## preliminaries

In [6]:
import nipype.interfaces.io as nio           # Data i/o
import nipype.interfaces.spm as spm          # spm
import nipype.interfaces.matlab as mlab      # how to run matlab
import nipype.interfaces.utility as util     # utility
import nipype.pipeline.engine as pe          # pypeline engine
import nipype.algorithms.rapidart as ra      # artifact detection
import nipype.algorithms.modelgen as model   # model specification
import os                                    # system functions


In [7]:
spm_standalone='/homes_unix/hirsch/historique_fli_iam/essai_spm_stand_alone/spm12/run_spm12.sh'
mcr='/homes_unix/hirsch/historique_fli_iam/essai_spm_stand_alone/mcr2016/v91'

In [8]:
from nipype.interfaces import spm

matlab_cmd = ' '.join([spm_standalone,mcr,'batch','script'])
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

## Nipype verbosity

In [10]:
import nipype
# Optional: Use the following lines to increase verbosity of output
nipype.config.set('logging', 'workflow_level',  'INFO')
nipype.config.set('logging', 'interface_level', 'INFO')
nipype.logging.update_logging(nipype.config)

## 1 - we decode the xml file

In [11]:
flibasedir = '/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01'

In [12]:
# we get the xml file in this directory
def getXmlFile(flibasedir):
    import os
    dirs = os.listdir( flibasedir )
    #print dirs

    import re
    xmlmatch = r'(.*)xml'
    for dir in dirs:
        xmlfound=re.search(xmlmatch,dir,flags=0)
        if xmlfound:
            #print xmlfound.group()
            return flibasedir + '/' + xmlfound.group()

In [13]:
xmlfile = getXmlFile(flibasedir)

print xmlfile

/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/xml_example.xml


In [14]:
# the we decode the xmlfile
import xml.etree.ElementTree as ET
tree = ET.parse(xmlfile)
root = tree.getroot()

In [15]:
print root.tag

def getEpiInfos(xmlfile):
    result = {}
    try:
        import xml.etree.ElementTree as ET
        tree = ET.parse(xmlfile)
        root = tree.getroot()
        # epibold directory
        for epibold in root.iter('EPIBOLD'):
            for child in epibold:
                print child.text
                if child.tag == 'file':
                    epidir = child.text
                    result['epidir'] = epidir
                    print epidir
                    
                # parameters
                if child.tag == 'parameters':
                    print "parameters"
                    
                    for child2 in child:
                                                     
                        # TR
                        if child2.tag == 'TR':
                            for child3 in child2:
                                if child3.tag == 'value':
                                    tr = child3.text
                                    print tr
                                    result['TR'] = tr
                    
                        # dynamics
                        if child2.tag == 'dynamics':
                            for child3 in child2:
                                if child3.tag == 'value':
                                    dynamics = child3.text
                                    result['dynamics'] = dynamics
                                    
                        # sliceTimingVector
                        if child2.tag == 'sliceTimingVector':
                            for child3 in child2:
                                if child3.tag == 'value':
                                    sliceTimingVector = child3.text
                                    result['sliceTimingVector'] = sliceTimingVector
                                    
                        # nb_slices
                        if child2.tag == 'nb_slices':
                            for child3 in child2:
                                if child3.tag == 'value':
                                    nb_slices = child3.text
                                    result['nb_slices'] = nb_slices
                
        
    except:
        print 'exception'
    return result


RS_analysis


In [16]:
epiResults = getEpiInfos(xmlfile)
print epiResults

/EPIBOLD/
/EPIBOLD/
bold-dataset

     
parameters
2
{'sliceTimingVector': '1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 2 4 6 8 10 12 14 16 18 20 22 24 26 28 30', 'dynamics': '240', 'nb_slices': '31', 'TR': '2', 'epidir': '/EPIBOLD/'}


In [17]:
def getT1file(xmlfile):
    try:
        import xml.etree.ElementTree as ET
        tree = ET.parse(xmlfile)
        root = tree.getroot()
        # T1 directory and file
        for t1 in root.iter('T1'):
            for child in t1:
                #print child.text
                if child.tag == 'file':
                    t1 = child.text
                    t1 = flibasedir + t1
                    # print  t1
    except:
        print 'exception'
    return t1
    
    

In [18]:
t1 = getT1file(xmlfile)
print t1

/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/T1/t0009_t1_s03.nii


# Pileline declaration

In [19]:
preproc = pe.Workflow(name='preproc')

## first node data grabbing by selcet files 

In [20]:
epidir = getEpiInfos(xmlfile)['epidir']
epidir = flibasedir + epidir

from nipype import SelectFiles, Node
templates = dict(T1=t1,
                 epi= epidir + "/" + "*.nii")

filesource = Node(SelectFiles(templates), "filesource")
filesource.inputs.subject_id = "subj1"
filesource.outputs.get()


/EPIBOLD/
/EPIBOLD/
bold-dataset

     
parameters
2


{'T1': <undefined>, 'epi': <undefined>}

## segment T1

In [25]:
segment = pe.Node(interface=spm.NewSegment(), name='segment')

#seg.inputs.channel_files = 't0009_t1_s03.nii'
segment.inputs.channel_info =(0.001,60.0,(True,True))          
segment.inputs.sampling_distance= 3.0
# todo donner un chemin de TPM.nii comme argument de la future fct
tissue1 = (('/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/Atlases/TPM.nii', 1), 1, (True,False), (False, False))
tissue2 = (('/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/Atlases/TPM.nii', 2), 1, (True,False), (False, False))
tissue3 = (('/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/Atlases/TPM.nii', 3), 2, (True,False), (False, False))
tissue4 = (('/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/Atlases/TPM.nii', 4), 3, (True,False), (False, False))
tissue5 = (('/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/Atlases/TPM.nii', 5), 4, (True,False), (False, False))
tissue6 = (('/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/Atlases/TPM.nii', 6), 2, (False,False), (False, False))
segment.inputs.tissues = [tissue1, tissue2, tissue3, tissue4, tissue5, tissue6]
segment.inputs.affine_regularization = 'mni'
#segment.inputs.warping_regularization = [0.0, 0.001, 0.5, 0.05, 0.2]
segment.inputs.write_deformation_fields = [False, False]

# connection du pipeline

In [26]:
preproc.connect(filesource,"T1" ,segment, 'channel_files')

## datasink

In [27]:
datasink = pe.Node(nio.DataSink(), name='datasink')
datasink.inputs.base_directory = '/homes_unix/hirsch/_pypipe/datadir/data_results'

In [28]:
preproc.connect(segment,  'normalized_class_images', datasink, 'structural')
preproc.connect(segment,  'bias_corrected_images' , datasink, 'structural.@par')
preproc.connect(segment,  'bias_field_images' , datasink, 'structural.@par1')
preproc.connect(segment,  'forward_deformation_field' , datasink, 'structural.@par2')
preproc.connect(segment,  'inverse_deformation_field' , datasink, 'structural.@par3')
preproc.connect(segment,  'modulated_class_images' , datasink, 'structural.@par4')
preproc.connect(segment,  'native_class_images' , datasink, 'structural.@par5')
preproc.connect(segment,  'transformation_mat' , datasink, 'structural.@par6')
preproc.connect(segment,  'dartel_input_images' , datasink, 'structural.@par7')

In [29]:
preproc.run()

INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running serially.
INFO:workflow:Executing node filesource in dir: /tmp/tmpYQYnM9/preproc/filesource
INFO:workflow:Runtime memory and threads stats unavailable
INFO:workflow:Executing node segment in dir: /tmp/tmpgJ0PZC/preproc/segment
INFO:workflow:Runtime memory and threads stats unavailable
INFO:workflow:Executing node datasink in dir: /tmp/tmpsPcrIg/preproc/datasink
INFO:workflow:Runtime memory and threads stats unavailable


# Brouillons

In [ ]:
bias_corrected_images = <undefined>
bias_field_images = <undefined>
dartel_input_images = <undefined>
forward_deformation_field = <undefined>
inverse_deformation_field = <undefined>
modulated_class_images = <undefined>
native_class_images = <undefined>
normalized_class_images = <undefined>
transformation_mat = <undefined>

In [ ]:
#advanced way to connect multiple nodes
workflowname.connect([(nodename1, nodename2, [('output_node1', 'input_node2')]),
                      (nodename1, nodename3, [('output_node1', 'input1_node3')]),
                      (nodename2, nodename3, [('output1_node2', 'input1_node3'),
                                              ('output2_node2', 'input2_node3')
                                              ])
                      ])

In [ ]:
preproc.connect([(filesource, segment, [('T1', 'channel_files')]),
                      (segment, datasink, [('normalized_class_images', 'normalized'),
                                              ('bias_corrected_images', 'bias_corrected')
                                              ])
                      ])

In [ ]:
datasink = pe.Node(nio.DataSink(), name='datasink')
datasink.inputs.base_directory = '/homes_unix/hirsch/_pypipe/datadir/data_results'
preproc.connect(segment,  'normalized_class_images', datasink, 'structural')
preproc.connect(segment,  'bias_corrected_images' , datasink, 'structural.@par')


In [ ]:
preproc.run()

In [4]:
spm.NewSegment.help()

Use spm_preproc8 (New Segment) to separate structural images into different
tissue classes. Supports multiple modalities.

NOTE: This interface currently supports single channel input only

http://www.fil.ion.ucl.ac.uk/spm/doc/manual.pdf#page=43

Examples
--------
>>> import nipype.interfaces.spm as spm
>>> seg = spm.NewSegment()
>>> seg.inputs.channel_files = 'structural.nii'
>>> seg.inputs.channel_info = (0.0001, 60, (True, True))
>>> seg.run() # doctest: +SKIP

For VBM pre-processing [http://www.fil.ion.ucl.ac.uk/~john/misc/VBMclass10.pdf],
TPM.nii should be replaced by /path/to/spm8/toolbox/Seg/TPM.nii

>>> seg = NewSegment()
>>> seg.inputs.channel_files = 'structural.nii'
>>> tissue1 = (('TPM.nii', 1), 2, (True,True), (False, False))
>>> tissue2 = (('TPM.nii', 2), 2, (True,True), (False, False))
>>> tissue3 = (('TPM.nii', 3), 2, (True,False), (False, False))
>>> tissue4 = (('TPM.nii', 4), 2, (False,False), (False, False))
>>> tissue5 = (('TPM.nii', 5), 2, (False,False), (False, Fa

In [3]:
spm.NewSegment.input_spec()


affine_regularization = <undefined>
channel_files = <undefined>
channel_info = <undefined>
ignore_exception = False
matlab_cmd = <undefined>
mfile = True
paths = <undefined>
sampling_distance = <undefined>
tissues = <undefined>
use_mcr = <undefined>
use_v8struct = True
warping_regularization = <undefined>
write_deformation_fields = <undefined>

In [5]:
spm.NewSegment.output_spec()


bias_corrected_images = <undefined>
bias_field_images = <undefined>
dartel_input_images = <undefined>
forward_deformation_field = <undefined>
inverse_deformation_field = <undefined>
modulated_class_images = <undefined>
native_class_images = <undefined>
normalized_class_images = <undefined>
transformation_mat = <undefined>

In [ ]:
/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/Atlases/TPM

In [ ]:
#Where should the output data be stored at?
sink = nipype.DataSink()
sink.inputs.base_directory = experiment_dir + '/output_folder'

In [ ]:
%cd /homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/EPIBOLD
%pwd